# summary of Air Quanlity data
## Dataset of "pm25_frm_daily_summary"
### (1) there are 301902 data items for PA
### (2) from 1999-01-01 to 2017-12-31	

In [37]:
# map the zip code to the longitude and latitude 
import numpy as np
import uszipcode
global sarchEngine 
searchEngine = uszipcode.SearchEngine()
def GetZipCodeInfo(zipCode):
    global searchEngine
    myZip = searchEngine.by_zipcode(zipCode)
    population,lat,lng = [myZip.population,myZip.lat,myZip.lng] # population info may need further updated later
    return population,lat,lng
print(GetZipCodeInfo("10001"))

(21102, 40.75, -73.99)


In [40]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from sklearn import linear_model
import sys
from collections import Counter

In [54]:
import math
def EuDist(locA,locB):
    x1,y1 = locA
    x2,y2 = locB
    return math.sqrt((x1-x2)**2+(y1-y2)**2)

# return the nearest zipCode according to loc
def NearZipCode(loc,zipDict):
    minDist = 1000000000 # set it a huge value
    minZip = ""
    for tmpZip in zipDict.keys():
        tmpLoc = zipDict[tmpZip]
        tmpDist = EuDist(tmpLoc,loc)
        if tmpDist<minDist:
            minDist = tmpDist
            minZip = tmpZip
    if minZip == "":
        print("err in NearZipcode")
    return minZip,minDist

def getZipLst():
#     here return a list of zipcode that we cares about
    return ["15211","10001"]

# from a list of zipcodes, return a diction [zipcode:loc]
def getZipDict(zipLst):
    r = {}
    for zipCode in zipLst:
        population,lat,lng = GetZipCodeInfo(zipCode)
        r[zipCode] = [lat,lng]
    return r
r = getZipDict(getZipLst())
print(r)
test_loc = [44.4,-80]
print("!",NearZipCode(test_loc,r))

{'15211': [40.43, -80.02], '10001': [40.75, -73.99]}
! ('15211', 3.970050377514118)


In [19]:
import pickle
# load sorted real data {zipcode:[data of visiting]}, data of visiting is in format "Day-Month-Year"
def LoadDict(p="../result/dict.pkl"):
    return pickle.load(open(p))

In [58]:
# read aqi data
import csv
global aqiDir
aqiDir = "../data/aqi/"
# return the average aqi for each month, of each loc
def GetAqiAve(Month):
    global aqiDir
    r = {}
    p = aqiDir + "aqi_{}.csv".format(Month)
    print(p)
    with open(p) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        idx = 0
        for row in csv_reader:  
            idx = idx+1
            if idx == 1:
                continue
            lat,lgn,_,aqi = row
            lat = float(lat)
            lgn = float(lgn)
            loc = (lat,lgn)
            if loc not in r:
                r[loc] = []
            r[loc].append(int(aqi))
    ret = {}
    for loc in r.keys():
        ret[loc] = np.array(r[loc]).mean()
            
    return ret
# return a dict of average for the zipcode in zip list
def GetAqiZipDict(zipLst,aqiDict):
    zipDict = getZipDict(zipLst)
    r = {}
    for loc in aqiDict:
        tmpZip,d = NearZipCode(loc,zipDict)
        if tmpZip not in r:
            r[tmpZip] = []
        r[tmpZip].append(aqiDict[loc])
    return r

zipLst = getZipLst()
aqiDict = GetAqiAve(1)
r = GetAqiZipDict(zipLst,aqiDict)
print(r)



../data/aqi/aqi_1.csv
{'10001': [44.0, 40.53448275862069, 34.93103448275862, 29.357142857142858, 42.26923076923077, 44.08, 46.0, 38.54237288135593, 44.2, 50.86440677966102, 40.22857142857143, 40.666666666666664, 40.53333333333333, 44.36, 49.0, 41.25, 28.0625, 43.38461538461539, 39.70175438596491, 44.416666666666664, 34.3125], '15211': [26.466666666666665, 39.3, 32.13333333333333, 29.352941176470587, 44.22222222222222, 48.93333333333333, 29.166666666666668, 36.06666666666667, 38.666666666666664, 33.75, 56.13333333333333, 38.666666666666664, 33.75, 46.51020408163265, 41.193548387096776, 45.03333333333333, 47.1, 34.0, 32.644444444444446, 34.65, 26.666666666666668, 31.771929824561404, 21.133333333333333]}


In [42]:
# get asthma result 
datafile1 = open('../data/results-20181005-221531.csv', 'r') 
f= pd.read_csv(datafile1)

f.head()
f['more_than_one_encounter'] = f['more_than_one_encounter'].astype(object)

f.head()

,visit_number,unique_patient_id,discharge_date,Time,Day,age,zip_code,race,sex,more_than_one_encounter
0,328302775314,840049133,10/23/2017,8:05:00,PM,21,15147,Black,Female,False
1,254421805169,981115173,6/2/2017,10:56:00,AM,21,15206,Black,Female,False
2,254421995290,981115172,9/30/2017,2:44:00,AM,21,15206,Black,Female,False
3,880369055290,785542959,9/29/2017,7:47:00,PM,50,15206,Black,Female,False
4,406633045281,980953611,9/20/2017,11:06:00,PM,25,15212,Black,Female,False
